In [8]:
import pandas as pd
import datetime
import numpy as np

In [9]:
df = pd.read_csv('group1_final-2.csv', index_col=0)

In [10]:
df.rename(columns = {'timestamp':'dt', 'userid': 'uid'}, inplace = True)

In [42]:
df['action'].unique()

array(['product', 'mainpage', 'category', 'search', 'cart', 'checkout',
       'confirmation'], dtype=object)

In [12]:
df['id'] = df.index

In [14]:
def swap_columns (df, col1, col2):
    col_list = list(df.columns )
    x, y = col_list. index (col1), col_list. index (col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [15]:
df = swap_columns (df, 'id', 'dt')

In [17]:
df = swap_columns (df, 'id', 'uid')

In [18]:
df

,id,uid,action,value,category,age,gender,city,dt
0,0,user_10366,product,0.0,Авто/мото товары,47,Ж,Уфа,2021-12-26T21:30:13
1,1,user_10467,mainpage,0.0,None,16,Ж,Москва,2021-12-26T21:30:14
2,2,user_10366,category,0.0,Авто/мото товары,47,Ж,Уфа,2021-12-26T21:30:16
3,3,user_10467,search,0.0,Товары для детей,16,Ж,Москва,2021-12-26T21:30:43
4,4,user_10366,mainpage,0.0,None,47,Ж,Уфа,2021-12-26T21:30:44
...,...,...,...,...,...,...,...,...,...
515593,515593,user_10418,category,0.0,Продовольственные товары,55,М,Новосибирск,2023-01-02T20:48:58
515594,515594,user_10418,category,0.0,Одежда и обувь,55,М,Новосибирск,2023-01-02T20:49:04
515595,515595,user_10418,search,0.0,Электроника и бытовая техника,55,М,Новосибирск,2023-01-02T20:49:09
515596,515596,user_10418,product,0.0,Красота и здоровье,55,М,Новосибирск,2023-01-02T20:49:47


In [19]:
def from_bad_time_to_good(date_time_str):
    return datetime.datetime.strptime(date_time_str, '%Y-%m-%dT%H:%M:%S')

In [20]:
df['dt'] = df['dt'].apply(from_bad_time_to_good)

In [21]:
df = df.sort_values(['uid', 'dt'])

In [23]:
df['diff'] = df.groupby(['uid'])['dt'].diff(1)

In [25]:
sessions_start_df = df[(df['diff'].isnull()) | (df['diff'] > pd.Timedelta("1800 seconds"))]

In [27]:
sessions_start_df['session_id'] = sessions_start_df['id']

C:\Users\Кирилл\AppData\Local\Temp\ipykernel_1532\2821468631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessions_start_df['session_id'] = sessions_start_df['id']


In [33]:
sessions_start_df

,id,uid,action,value,category,age,gender,city,dt,diff,session_id
0,0,user_10366,product,0.0,Авто/мото товары,47,Ж,Уфа,2021-12-26 21:30:13,NaT,0
1,1,user_10467,mainpage,0.0,None,16,Ж,Москва,2021-12-26 21:30:14,NaT,1
15,15,user_10440,mainpage,0.0,None,47,Ж,Нижний Новгород,2021-12-26 22:04:04,NaT,15
20,20,user_10268,search,0.0,Красота и здоровье,51,Ж,Омск,2021-12-26 22:30:44,NaT,20
21,21,user_10255,mainpage,0.0,None,14,М,Москва,2021-12-26 22:35:59,NaT,21
...,...,...,...,...,...,...,...,...,...,...,...
515549,515549,user_10008,search,0.0,Электроника и бытовая техника,23,Ж,Нижний Новгород,2023-01-02 20:19:50,3 days 00:00:04,515549
515563,515563,user_10017,category,0.0,Красота и здоровье,28,Ж,Самара,2023-01-02 20:22:33,1 days 00:00:04,515563
515569,515569,user_10274,category,0.0,Электроника и бытовая техника,67,М,Москва,2023-01-02 20:23:12,4 days 00:00:20,515569
515582,515582,user_10303,search,0.0,Авто/мото товары,75,Ж,Уфа,2023-01-02 20:43:51,2 days 00:00:22,515582


In [29]:
df = df.sort_values('id')
sessions_start_df = sessions_start_df.sort_values('id')
df = pd.merge_asof(df, sessions_start_df[['uid','id','session_id']],on='id',by='uid')

In [30]:
df

,id,uid,action,value,category,age,gender,city,dt,diff,session_id
0,0,user_10366,product,0.0,Авто/мото товары,47,Ж,Уфа,2021-12-26 21:30:13,NaT,0
1,1,user_10467,mainpage,0.0,None,16,Ж,Москва,2021-12-26 21:30:14,NaT,1
2,2,user_10366,category,0.0,Авто/мото товары,47,Ж,Уфа,2021-12-26 21:30:16,0 days 00:00:03,0
3,3,user_10467,search,0.0,Товары для детей,16,Ж,Москва,2021-12-26 21:30:43,0 days 00:00:29,1
4,4,user_10366,mainpage,0.0,None,47,Ж,Уфа,2021-12-26 21:30:44,0 days 00:00:28,0
...,...,...,...,...,...,...,...,...,...,...,...
515593,515593,user_10418,category,0.0,Продовольственные товары,55,М,Новосибирск,2023-01-02 20:48:58,0 days 00:00:29,515590
515594,515594,user_10418,category,0.0,Одежда и обувь,55,М,Новосибирск,2023-01-02 20:49:04,0 days 00:00:06,515590
515595,515595,user_10418,search,0.0,Электроника и бытовая техника,55,М,Новосибирск,2023-01-02 20:49:09,0 days 00:00:05,515590
515596,515596,user_10418,product,0.0,Красота и здоровье,55,М,Новосибирск,2023-01-02 20:49:47,0 days 00:00:38,515590


In [31]:
df.sort_values('session_id')

,id,uid,action,value,category,age,gender,city,dt,diff,session_id
0,0,user_10366,product,0.0,Авто/мото товары,47,Ж,Уфа,2021-12-26 21:30:13,NaT,0
14,14,user_10366,search,0.0,Авто/мото товары,47,Ж,Уфа,2021-12-26 21:33:07,0 days 00:01:04,0
12,12,user_10366,category,0.0,Авто/мото товары,47,Ж,Уфа,2021-12-26 21:31:45,0 days 00:00:13,0
10,10,user_10366,mainpage,0.0,None,47,Ж,Уфа,2021-12-26 21:31:32,0 days 00:00:27,0
7,7,user_10366,search,0.0,Одежда и обувь,47,Ж,Уфа,2021-12-26 21:31:05,0 days 00:00:18,0
...,...,...,...,...,...,...,...,...,...,...,...
515596,515596,user_10418,product,0.0,Красота и здоровье,55,М,Новосибирск,2023-01-02 20:49:47,0 days 00:00:38,515590
515591,515591,user_10418,search,0.0,Продовольственные товары,55,М,Новосибирск,2023-01-02 20:48:16,0 days 00:00:20,515590
515590,515590,user_10418,search,0.0,Продовольственные товары,55,М,Новосибирск,2023-01-02 20:47:56,1 days 00:00:24,515590
515592,515592,user_10418,mainpage,0.0,None,55,М,Новосибирск,2023-01-02 20:48:29,0 days 00:00:13,515590


In [32]:
df

,id,uid,action,value,category,age,gender,city,dt,diff,session_id
0,0,user_10366,product,0.0,Авто/мото товары,47,Ж,Уфа,2021-12-26 21:30:13,NaT,0
1,1,user_10467,mainpage,0.0,None,16,Ж,Москва,2021-12-26 21:30:14,NaT,1
2,2,user_10366,category,0.0,Авто/мото товары,47,Ж,Уфа,2021-12-26 21:30:16,0 days 00:00:03,0
3,3,user_10467,search,0.0,Товары для детей,16,Ж,Москва,2021-12-26 21:30:43,0 days 00:00:29,1
4,4,user_10366,mainpage,0.0,None,47,Ж,Уфа,2021-12-26 21:30:44,0 days 00:00:28,0
...,...,...,...,...,...,...,...,...,...,...,...
515593,515593,user_10418,category,0.0,Продовольственные товары,55,М,Новосибирск,2023-01-02 20:48:58,0 days 00:00:29,515590
515594,515594,user_10418,category,0.0,Одежда и обувь,55,М,Новосибирск,2023-01-02 20:49:04,0 days 00:00:06,515590
515595,515595,user_10418,search,0.0,Электроника и бытовая техника,55,М,Новосибирск,2023-01-02 20:49:09,0 days 00:00:05,515590
515596,515596,user_10418,product,0.0,Красота и здоровье,55,М,Новосибирск,2023-01-02 20:49:47,0 days 00:00:38,515590


In [36]:
df = df.drop(['diff'], axis=1)

In [37]:
df

,id,uid,action,value,category,age,gender,city,dt,session_id
0,0,user_10366,product,0.0,Авто/мото товары,47,Ж,Уфа,2021-12-26 21:30:13,0
1,1,user_10467,mainpage,0.0,None,16,Ж,Москва,2021-12-26 21:30:14,1
2,2,user_10366,category,0.0,Авто/мото товары,47,Ж,Уфа,2021-12-26 21:30:16,0
3,3,user_10467,search,0.0,Товары для детей,16,Ж,Москва,2021-12-26 21:30:43,1
4,4,user_10366,mainpage,0.0,None,47,Ж,Уфа,2021-12-26 21:30:44,0
...,...,...,...,...,...,...,...,...,...,...
515593,515593,user_10418,category,0.0,Продовольственные товары,55,М,Новосибирск,2023-01-02 20:48:58,515590
515594,515594,user_10418,category,0.0,Одежда и обувь,55,М,Новосибирск,2023-01-02 20:49:04,515590
515595,515595,user_10418,search,0.0,Электроника и бытовая техника,55,М,Новосибирск,2023-01-02 20:49:09,515590
515596,515596,user_10418,product,0.0,Красота и здоровье,55,М,Новосибирск,2023-01-02 20:49:47,515590


515598

In [38]:
df.to_csv('actions_sessions.csv', index=False)